In [2]:
pwd

'/home/wsuser/work'

In [3]:
!pip install keras==2.2.4
!pip install tensorflow==1.14.0

/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |████████████████████████████████| 312 kB 13.9 MB/s eta 0:00:01
/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |█████████████████

In [4]:
from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.models import Sequential
from keras.layers import Conv2D , MaxPooling2D
from keras.layers import Activation, Flatten, Dropout, Dense, BatchNormalization
from keras import backend as K
import os
import numpy as np
import pandas as pd

Using TensorFlow backend.
/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/envs/Python-

In [10]:
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.

if os.environ.get('RUNTIME_ENV_LOCATION_TYPE') == 'external':
    endpoint_921f7d4a17b746bebb27bcb4d0619ead = 'https://s3.eu.cloud-object-storage.appdomain.cloud'
else:
    endpoint_921f7d4a17b746bebb27bcb4d0619ead = 'https://s3.private.eu.cloud-object-storage.appdomain.cloud'

client_921f7d4a17b746bebb27bcb4d0619ead = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='-kUuVaE4zyFeGlwigyp0asAPz60B5zhaITfwWTXMCgB5',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url=endpoint_921f7d4a17b746bebb27bcb4d0619ead)

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/
streaming_body_1 = client_921f7d4a17b746bebb27bcb4d0619ead.get_object(Bucket='pneumonia-donotdelete-pr-uzaval0izfmlng', Key='chest_xray.zip')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(streaming_body_1, "__iter__"): streaming_body_1.__iter__ = types.MethodType( __iter__, streaming_body_1 ) 


In [11]:
from io import BytesIO
import zipfile
unzip = zipfile.ZipFile(BytesIO(streaming_body_1.read()),'r')
file_paths = unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [12]:
pwd

'/home/wsuser/work'

In [13]:
import os
filenames = os.listdir('/home/wsuser/work/chest_xray/train')

In [14]:
train_datagen = ImageDataGenerator(rescale = 1./255, shear_range = 0.2, zoom_range = 0.2, rotation_range = 30,
                                   width_shift_range=0.1, height_shift_range=0.1, horizontal_flip = True, vertical_flip=False)
test_datagen = ImageDataGenerator(rescale = 1./255)

In [15]:
train_gen = train_datagen.flow_from_directory('/home/wsuser/work/chest_xray/train',
                                            target_size = (150,150),
                                            batch_size = 32,
                                            class_mode = "binary" )

Found 5216 images belonging to 2 classes.


In [16]:
test_gen = train_datagen.flow_from_directory('/home/wsuser/work/chest_xray/test',
                                            target_size = (150,150),
                                            batch_size = 32,
                                            class_mode = "binary" )

Found 624 images belonging to 2 classes.


In [17]:
val_gen = train_datagen.flow_from_directory('/home/wsuser/work/chest_xray/val',
                                            target_size = (150,150),
                                            batch_size = 32,
                                            class_mode = "binary" )

Found 16 images belonging to 2 classes.


In [18]:
train_gen.class_indices

{'NORMAL': 0, 'PNEUMONIA': 1}

In [19]:
model = Sequential()
# The number of filters are 32 and the kernal_size is (3,3)
model.add(Conv2D(32, (3, 3), input_shape=(150,150,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dropout(0.2))

model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dense(1))
model.add(Activation('sigmoid'))






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [20]:
model.compile(optimizer = "adam",loss = "binary_crossentropy",metrics = ["acc"])



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [21]:
history = model.fit_generator(train_gen,
                         steps_per_epoch = 163,
                         epochs = 20,
                         validation_data = val_gen,
                         validation_steps = 1)

Epoch 1/20
163/163 [==============================] - 226s 1s/step - loss: 0.5413 - acc: 0.7487 - val_loss: 0.7230 - val_acc: 0.5625
Epoch 2/20
163/163 [==============================] - 220s 1s/step - loss: 0.3874 - acc: 0.8230 - val_loss: 0.7878 - val_acc: 0.7500
Epoch 3/20
163/163 [==============================] - 226s 1s/step - loss: 0.3259 - acc: 0.8585 - val_loss: 0.3922 - val_acc: 0.8125
Epoch 4/20
163/163 [==============================] - 220s 1s/step - loss: 0.2575 - acc: 0.8967 - val_loss: 1.0366 - val_acc: 0.5625
Epoch 5/20
163/163 [==============================] - 219s 1s/step - loss: 0.2288 - acc: 0.9062 - val_loss: 1.1226 - val_acc: 0.6250
Epoch 6/20
163/163 [==============================] - 220s 1s/step - loss: 0.2220 - acc: 0.9101 - val_loss: 2.5721 - val_acc: 0.5625
Epoch 7/20
163/163 [==============================] - 217s 1s/step - loss: 0.2236 - acc: 0.9112 - val_loss: 2.0291 - val_acc: 0.5000
Epoch 8/20
163/163 [==============================] - 218s 1s/step - 

In [22]:
#Accuracy of test data.
test_acc = model.evaluate_generator(test_gen,624/32)
print("\nAccuracy:"+" %.2f%%" % ( test_acc[1]*100))


Accuracy: 88.46%


In [24]:
model.save("pneumonia.h5")

In [25]:
!tar -zcvf image-classification-model_new.tgz pneumonia.h5

pneumonia.h5


In [26]:
ls -1

chest_xray/
image-classification-model_new.tgz
pneumonia.h5


In [27]:
!pip install watson-machine-learning-client --upgrade

/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |████████████████████████████████| 538 kB 14.4 MB/s eta 0:00:01


In [28]:
#Replace the credentials you got from Watson ML Service
from ibm_watson_machine_learning import APIClient
wml_credentials =   {
                        "url" : "https://us-south.ml.cloud.ibm.com",
                        "apikey" : "agryfSUNw18FnbjLIu47mROvPKgW3RLZBtRpuytF1Zw5"
                    }
client = APIClient(wml_credentials)

In [31]:
def guid_from_space_name(client, space_name):
    space = client.spaces.get_details()
    #print(space)
    return (next(item for item in space['resources'] if item['entity']["name"] == space_name)['metadata']['id'])

In [32]:
space_uid = guid_from_space_name(client, 'pneumonia')
print("Space UID = "+ space_uid)

Space UID = 969eed2e-4acf-46ca-a989-3bb879c7abe3


In [33]:
client.set.default_space(space_uid)

'SUCCESS'

In [34]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
scikit-learn_0.22-py3.6        154010fa-5b3b-4ac1-82af-4d5ee5abbc85  base
default_r3.6                   1b70aec3-ab34-4b87-8aa0-a4a3c8296a36  base
pytorch-onnx_1.3-py3.6         1bc6029

In [35]:
software_spec_uid = client.software_specifications.get_uid_by_name("tensorflow_1.15-py3.6")
software_spec_uid

'2b73a275-7cbf-420b-a912-eae7f436e0bc'

In [36]:
model_details =  client.repository.store_model(model='image-classification-model_new.tgz',
                meta_props={
                    client.repository.ModelMetaNames.NAME:"CNN",
                    client.repository.ModelMetaNames.TYPE:"keras_2.2.4",
                    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_spec_uid
                })
model_id = client.repository.get_model_uid(model_details)

Note: Warnings!! :  Model type keras_2.2.4 is deprecated. We recommend you use a supported model type. See Supported Frameworks https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/pm_service_supported_frameworks.html


In [37]:
model_id

'5871d95e-38db-4ccb-9f47-24091a4a2183'

In [38]:
client.repository.download(model_id, 'my_model.tar.gz')

Successfully saved model content to file: 'my_model.tar.gz'


'/home/wsuser/work/my_model.tar.gz'

In [39]:
model_json = model.to_json()
with open("model.json","w") as json_file:
    json_file.write(model_json)

In [40]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
import cv2

In [41]:
model = load_model("pneumonia.h5")

In [42]:
from skimage.transform import resize

In [43]:
def detect(frame):
    try:
        img = resize(frame, (150,150))
        img = np.expand_dims(img, axis=0)
        prediction = model.predict(img)
        print(prediction)
        prediction = prediction > 0.5
        print(prediction)
        s = "PNEUMONIA" if (prediction[0][0]) else "NORMAL"
        return (s)
    except AttributeError:
        print("SHAPE NOT FOUND")

In [63]:

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/
streaming_body_2 = client_921f7d4a17b746bebb27bcb4d0619ead.get_object(Bucket='pneumonia-donotdelete-pr-uzaval0izfmlng', Key='test1.jpeg')['Body']
# add missing __iter__ method so pandas accepts body as file-like object
if not hasattr(streaming_body_2, "__iter__"): streaming_body_2.__iter__ = types.MethodType( __iter__, streaming_body_2 ) 


In [64]:
img = image.load_img(streaming_body_2,target_size = (150,150))

In [65]:
x = image.img_to_array(img)
x = np.expand_dims(img, axis=0)
pred = model.predict_classes(x)

In [68]:
pred[0][0]

0

In [69]:

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/
streaming_body_3 = client_921f7d4a17b746bebb27bcb4d0619ead.get_object(Bucket='pneumonia-donotdelete-pr-uzaval0izfmlng', Key='test2.jpeg')['Body']
# add missing __iter__ method so pandas accepts body as file-like object
if not hasattr(streaming_body_3, "__iter__"): streaming_body_3.__iter__ = types.MethodType( __iter__, streaming_body_3 ) 


In [71]:
img = image.load_img(streaming_body_3,target_size = (150,150))

In [72]:
y = image.img_to_array(img)
y = np.expand_dims(img, axis=0)
pred = model.predict_classes(y)

In [73]:
pred[0][0]

0